In [217]:
## IMPORTS ##
import pandas as pd
from recordlinkage import Compare, Index
from datetime import datetime
import numpy as np
import re

In [218]:
# Load anonymized and external datasets
anonymized_df = pd.read_csv('anonymised_dataE.csv')
register_df = pd.read_excel('public_data_registerE.xlsx')
results_df = pd.read_excel('public_data_resultsE.xlsx')
survey_names_df = pd.read_fwf('survey_listE.txt')

In [219]:
anonymized_df.head(50)

,Unnamed: 0,sex,evote,zip,education,citizenship,marital_status,party,age_group
0,1,Female,0,2400,Vocational bachelors educations,Denmark,Not married,Red,"(38,48]"
1,2,Male,1,2200,Masters programmes,Denmark,Married,Green,"(30,38]"
2,3,Female,1,2200,Vocational bachelors educations,Denmark,Not married,Green,"(18,30]"
3,4,Female,0,2200,Vocational Education and Training (VET),Denmark,Not married,Green,"(58,70]"
4,5,Female,0,2200,Vocational Education and Training (VET),Denmark,Not married,Green,"(58,70]"
5,6,Male,0,2300,Vocational bachelors educations,Germany,Not married,Green,"(30,38]"
6,7,Male,0,2100,Vocational Education and Training (VET),Denmark,Not married,Green,"(48,58]"
7,8,Male,0,2200,Upper secondary education,Denmark,Not married,Green,"(70,101]"
8,9,Female,1,2100,Bachelors programmes,Denmark,Not married,Red,"(38,48]"
9,10,Female,0,2100,Vocational Education and Training (VET),Denmark,Not married,Green,"(38,48]"


In [220]:
register_df.head()

,name,sex,dob,zip,citizenship,marital_status,last_voted
0,"Turner, Destiny",Female,2001-04-01,2300,Latvia,Never married,2
1,"Smith, Ian",Male,1999-07-26,2200,Denmark,Never married,2
2,"Tucker, Kateland",Female,2001-06-19,2300,Denmark,Never married,1
3,"Robles Talavera, Troy",Male,2000-02-03,2100,Denmark,Never married,2
4,"Cordova, Lexis",Female,1999-08-25,2200,Denmark,Never married,2


In [221]:
ls = anonymized_df['marital_status'].unique().tolist()
sorted(ls)

['Married', 'Not married']

In [222]:
ls1 = register_df['marital_status'].unique().tolist()
sorted(ls1)

['Divorced', 'Married/separated', 'Never married', 'Widowed']

In [223]:
# Assume 'date_of_birth' is the column with DoB in the format 'YYYY-MM-DD'
current_date = datetime.now()  # Use a specific date like datetime(2024, 1, 1) if needed

register_df['age'] = register_df['dob'].apply(
    lambda dob: current_date.year - pd.to_datetime(dob).year - 
    ((current_date.month, current_date.day) < (pd.to_datetime(dob).month, pd.to_datetime(dob).day))
)

In [224]:
# Age Grouping
register_df['age_group'] = pd.cut(
    register_df['age'], 
    bins=[18, 30, 38, 48, 58, 70, 101],  # Bin edges
    labels=['(18,30]', '(30,38]', '(38,48]', '(48,58]', '(58,70]', '(70,101]']  # Corresponding labels
)

In [225]:
# Marital Status Recoding
status_mapping = {
    'Divorced': 'Not married',
    'Married/separated': 'Married',
    'Never married': 'Not married',
    'Widowed': 'Not married'
}
register_df['marital_status'] = register_df['marital_status'].map(status_mapping)

In [226]:
# Simulate PRAM for Sex
#might remove it
np.random.seed(42)  # For reproducibility
register_df['pram_sex'] = register_df['sex'].apply(
    lambda x: x if np.random.rand() < 0.7 else ('Male' if x == 'Female' else 'Female')
)

In [227]:
register_df.head(10)

,name,sex,dob,zip,citizenship,marital_status,last_voted,age,age_group,pram_sex
0,"Turner, Destiny",Female,2001-04-01,2300,Latvia,Not married,2,23,"(18,30]",Female
1,"Smith, Ian",Male,1999-07-26,2200,Denmark,Not married,2,25,"(18,30]",Female
2,"Tucker, Kateland",Female,2001-06-19,2300,Denmark,Not married,1,23,"(18,30]",Male
3,"Robles Talavera, Troy",Male,2000-02-03,2100,Denmark,Not married,2,24,"(18,30]",Male
4,"Cordova, Lexis",Female,1999-08-25,2200,Denmark,Not married,2,25,"(18,30]",Female
5,"Rodriguez, Jafet",Male,2000-10-10,2200,Turkey,Not married,1,24,"(18,30]",Male
6,"al-Ansari, Mukarram",Male,2002-01-09,2100,Denmark,Not married,1,22,"(18,30]",Male
7,"el-Noorani, Huwaida",Female,2003-02-23,2200,Denmark,Not married,2,21,"(18,30]",Male
8,"Ramirez-Salaz, Jasmine",Female,2003-06-04,2400,Denmark,Not married,1,21,"(18,30]",Female
9,"Newsum Schoenberg, Emoni",Female,1999-08-14,2300,Denmark,Not married,1,25,"(18,30]",Male


In [228]:
# Indexing to reduce comparison space
indexer = Index()
indexer.block(['zip', 'age_group', 'marital_status'])  # Blocking on ziå to speed up
candidate_links = indexer.index(anonymized_df, register_df)

# Compare quasi-identifiers
compare = Compare()
compare.exact('zip', 'zip', label='zip')
compare.exact('age_group', 'age_group', label='age_group')
compare.exact('marital_status', 'marital_status', label='marital_status')
compare.exact('citizenship', 'citizenship', label='citizenship')

compare.string('sex', 'sex', method='levenshtein', label='sex')

comparison_results = compare.compute(candidate_links, anonymized_df, register_df)

# Sum scores across features
comparison_results['score'] = comparison_results.sum(axis=1)

# Select matches above a threshold
threshold = 4.5  # Adjust based on your confidence level
matches = comparison_results[comparison_results['score'] >= threshold]

# Extract matched anonymized indices and corresponding public register indices
matched_anonymized_indices = matches.index.get_level_values(0)  # Anonymized dataset indices
matched_register_indices = matches.index.get_level_values(1)  # External dataset indices

# Merge matched records with public register
matched_register = register_df.loc[matched_register_indices]
matched_anonymized = anonymized_df.loc[matched_anonymized_indices]

In [230]:
comparison_results

zip  age_group  marital_status  citizenship  sex  score
0   150    1          1               1            1  1.0    5.0
    159    1          1               1            1  0.5    4.5
    178    1          1               1            1  0.5    4.5
    187    1          1               1            1  0.5    4.5
    202    1          1               1            1  0.5    4.5
...      ...        ...             ...          ...  ...    ...
185 603    1          1               1            1  0.5    4.5
    625    1          1               1            1  0.5    4.5
    629    1          1               1            1  0.5    4.5
    638    1          1               1            1  0.5    4.5
    649    1          1               1            1  0.5    4.5

[8498 rows x 6 columns]

In [231]:
print(matched_anonymized)

     Unnamed: 0     sex  evote   zip                        education  \
0             1  Female      0  2400  Vocational bachelors educations   
0             1  Female      0  2400  Vocational bachelors educations   
0             1  Female      0  2400  Vocational bachelors educations   
0             1  Female      0  2400  Vocational bachelors educations   
0             1  Female      0  2400  Vocational bachelors educations   
..          ...     ...    ...   ...                              ...   
185         186    Male      0  2100                Primary education   
185         186    Male      0  2100                Primary education   
185         186    Male      0  2100                Primary education   
185         186    Male      0  2100                Primary education   
185         186    Male      0  2100                Primary education   

    citizenship marital_status  party age_group  
0       Denmark    Not married    Red   (38,48]  
0       Denmark    Not 

In [232]:
matched_register

,name,sex,dob,zip,citizenship,marital_status,last_voted,age,age_group,pram_sex
150,"al-Quadri, Sanad",Female,1983-11-22,2400,Denmark,Not married,2,40,"(38,48]",Male
159,"Sanchez, Caesar",Male,1981-11-02,2400,Denmark,Not married,1,43,"(38,48]",Female
178,"al-Momin, Sirajuddeen",Male,1979-08-05,2400,Denmark,Not married,1,45,"(38,48]",Female
187,"al-Wali, Humaid",Male,1981-09-20,2400,Denmark,Not married,1,43,"(38,48]",Male
202,"Kim, Abel",Male,1984-08-25,2400,Denmark,Not married,1,40,"(38,48]",Male
...,...,...,...,...,...,...,...,...,...,...
603,"Montoya, Celeste",Female,1995-05-13,2100,Denmark,Married,2,29,"(18,30]",Female
625,"Castinado, Eva",Female,1995-05-02,2100,Denmark,Married,0,29,"(18,30]",Female
629,"Haas, Jazmynn",Female,1994-02-19,2100,Denmark,Married,0,30,"(18,30]",Female
638,"Williams, Rachel",Female,1995-08-02,2100,Denmark,Married,0,29,"(18,30]",Female


In [233]:
#biased 65.5

# Step 1: Calculate probabilities per ZIP by first extracting ZIP codes and calculating the valid votes and probabilities
results_df['zip'] = results_df['zip'].str.extract(r'Polling station: ZIP (\d{4})')  # Extract the ZIP code
results_df = results_df.dropna(subset=['zip'])  # Drop rows with missing ZIP
results_df['zip'] = results_df['zip'].astype(int)  # Convert ZIP to integer

# Calculate total valid votes per ZIP
results_df['valid_votes'] = results_df['Red'] + results_df['Green']

# Calculate vote probabilities per ZIP (Red and Green)
results_df['party_red_zip'] = results_df.groupby('zip')['Red'].transform('sum') / results_df.groupby('zip')['valid_votes'].transform('sum')
results_df['party_green_zip'] = results_df.groupby('zip')['Green'].transform('sum') / results_df.groupby('zip')['valid_votes'].transform('sum')

# Step 2: Merge matched register with public results on zip to get ZIP-specific probabilities
matched_register_1 = matched_register.merge(results_df[['zip', 'party_red_zip', 'party_green_zip']], on='zip', how='left')

# Step 3: Assign the most probable party (Red or Green) based on the ZIP-specific probabilities
def assign_most_probable_party(row):
    # Compare the probabilities and assign the party with the highest probability
    if row['party_red_zip'] > row['party_green_zip']:
        return 'Red'  # Red is the most probable
    else:
        return 'Green'  # Green is the most probable

# Apply the function to assign the most probable political party for each person in matched register
matched_register_1['inferred_party'] = matched_register_1.apply(assign_most_probable_party, axis=1)

# Step 4: Combine inferred political preference with name
reidentified_data_1 = matched_register_1[['name', 'inferred_party']]

# Step 5: Filter the reidentified data to include only rows with names in survey_names_df
filtered_reidentified_data_1 = reidentified_data_1[reidentified_data_1['name'].isin(survey_names_df['name'])]

# Step 6: Aggregate the filtered data by name to get the most frequent political preference
filtered_reidentified_data_agg = filtered_reidentified_data_1.groupby('name').agg({
    'inferred_party': lambda x: x.mode()[0]  # Get the most frequent (mode) value
}).reset_index()

# Step 7: Save the reidentified data to CSV
filtered_reidentified_data_agg.to_csv('reidentified_data_E_1.csv', index=False)
